In [72]:
import pandas as pd
from sqlalchemy import create_engine

import plotly
import plotly.graph_objs as go
import plotly.express as px

import json

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import math

In [73]:
database_filepath = 'DisasterResponse.db'

In [74]:
# Create the engine
engine = create_engine(f"sqlite:///{database_filepath}")
# Load the data in a pandas Dataframe
df = pd.read_sql_table("disaster_messages", engine)

In [75]:
def get_graph_json(graph):
    return json.dumps(graph, cls=plotly.utils.PlotlyJSONEncoder)

# Example already in script

In [76]:
genre_counts = df.groupby('genre').count()['message']
genre_names = list(genre_counts.index)

# genre_counts_graph = px.bar(
#     x=genre_names,
#     y=genre_counts,
#     title='Distribution of Message Genres',
#     labels={
#         'x': 'Genre',
#         'y': 'Count'
#     }
# ).to_dict()

# plotly.io.from_json( get_graph_json(genre_counts_graph) )

# Counts per Message Category

In [77]:
# Counts per category for the top 10 categories
msg_category_cols = df.columns[4:]
cat_count = df[msg_category_cols].sum().sort_values(ascending=False).head(10).reset_index().rename(columns={'index': 'category', 0: 'count'})

# cat_counts_graph = {
#     'data': [
#         go.Bar(
#             x=msg_category_display,
#             y=cat_count
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Count of Messages by Category'
#         },
#         'yaxis': {
#             'title': {
#                 'text': "Count"
#             }
#         },
#         'xaxis': {
#             'title': {
#                 'text': "Message Category"
#             }
#         }
#     }
# }

cat_counts_graph = px.bar(
    data_frame=cat_count,
    x='category',
    y='count',
    title='Count of Messages by Category',
    labels={
        'x': 'Message Category',
        'y': 'Count'
    }
)

# plotly.io.from_json( get_graph_json( cat_counts_graph.to_dict() ) )

# Counts of translated message - Bar

In [78]:
# # df['diff_lang'] = df['message'] != df['original']
# df['diff_lang'] = df.apply( lambda row: 'Translated' if row['message'] != row['original'] else "English", axis=1 )
# diff_lang_count = df.groupby('diff_lang').count()['message'].reset_index().rename(columns={'message': 'count'})

# display(diff_lang_count)

# diff_lang_graph = {
#     'data': [
#         go.Bar(
#             x=diff_lang_count['diff_lang'],
#             y=diff_lang_count['count']
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Count of Messages Not in English'
#         },
#         'yaxis': {
#             'title': {
#                 'text': "Count"
#             }
#         },
#         'xaxis': {
#             'title': {
#                 'text': "Translation"
#             }
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(diff_lang_graph) )

# Count of translated messages - Pie

In [79]:
# df['diff_lang'] = df.apply( lambda row: 'Translated' if row['message'] != row['original'] else "English", axis=1 )
# diff_lang_count = df.groupby('diff_lang').count()['message'].reset_index().rename(columns={'message': 'count'})

# diff_lang_graph = {
#     'data': [
#         go.Pie(
#             labels=diff_lang_count['diff_lang'], values=diff_lang_count['count']
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Count of Messages Not in English'
#         }
#     }
# }

# plotly.io.from_json( get_graph_json(diff_lang_graph) )

# Counts of words per message - Pie chart

In [80]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [81]:
# Count of messages per length bin
bins = [0, 10, 20, 30, 40, 50, float('inf')]
labels=['0-9', '10-19', '20-29', '30-39', '40-49', '50+']
word_count = df['message'].apply( lambda msg: len(tokenize(msg)) ).rename('word_count').reset_index(drop=True)
word_count_bins = pd.cut( word_count, bins=bins, labels=labels )
count_per_bin = word_count_bins.value_counts().sort_index().reset_index()#.rename(columns={'word_count': 'Word Count In Message'})

# word_count_graph = {
#     'data': [
#         go.Pie(
#             labels=count_per_bin['bins'], values=count_per_bin['count'], sort=False
#         )
#     ],

#     'layout': {
#         'title': {
#             'text': 'Counts of Message Lengths'
#         }
#     }
# }

word_count_graph = px.pie(
    data_frame=count_per_bin,
    values='count', 
    names='word_count', 
    title='Counts of Message Lengths',
    labels={
        'word_count': 'Word Count',
        'count': 'Count'
    }
)

word_count_graph.update_traces(textinfo='percent+label')

# plotly.io.from_json( get_graph_json( word_count_graph.to_dict() ) )

# Counts of words per message - Histogram

In [82]:
# import plotly.figure_factory as ff

# word_count = df['message'].apply( lambda msg: len(tokenize(msg)) ).rename('word_count').reset_index(drop=True)
# word_count_u100 = word_count[word_count < 100]
# word_count_o100 = word_count[word_count >= 100]

# fig1 = ff.create_distplot([word_count_u100], ['Word Count (under 100 words)'], bin_size=1)
# fig2 = ff.create_distplot([word_count_o100], ['Word Count (over 100 words)'], bin_size=100)
# fig1.show()
# fig2.show()

# Message Categories by Count by Word Count

In [83]:
# group the dataframe by word_count bin and category and get the count for each group.
# x axis can be the word_count bin, y axis can be the count, and the size of the bubble can be the category count

# cat_wordcount_counts = df.copy()

# bins = [0, 10, 20, 30, 40, 50, float('inf')]
# labels=['0-9', '10-19', '20-29', '30-39', '40-49', '50+']
# word_count = cat_wordcount_counts['message'].apply( lambda msg: len(tokenize(msg)) ).rename('word_count').reset_index(drop=True)
# cat_wordcount_counts['word_count_bins'] = pd.cut( word_count, bins=bins, labels=labels )

# msg_category_cols = df.columns[4:]
# cat_count = cat_wordcount_counts[msg_category_cols].sum().sort_values(ascending=False)
# msg_category_display = [ cn.replace("_", " ").capitalize() for cn in cat_count.index ]

# cat_wordcount_counts = cat_wordcount_counts[['word_count_bins', *msg_category_cols]].groupby(['word_count_bins']).sum().reset_index()

# count_by_category_and_bin = pd.melt( cat_wordcount_counts, id_vars=['word_count_bins'], value_vars=msg_category_cols, var_name='category', value_name='count' )
# count_by_category_and_bin['marker_size'] = count_by_category_and_bin['word_count_bins'].apply( lambda x: int(x.split('-')[0]) + 10 if x != '50+' else 60 )

# total_counts = count_by_category_and_bin.groupby('category')['count'].sum().reset_index().rename(columns={'count': 'total_count'})

# count_by_category_and_bin = count_by_category_and_bin.merge(total_counts, on='category')
# count_by_category_and_bin['percentage'] = count_by_category_and_bin['count'] / count_by_category_and_bin['total_count'] * 100
# count_by_category_and_bin['percentage'] = count_by_category_and_bin['percentage'].apply( lambda x: round(x, 2) )

# count_by_category_and_bin.sort_values(by=['category', 'word_count_bins'], ascending=[True, False], inplace=True)


# ## creating a stacked barchart to show percentages per category - This is the final form, need to convert to json
# percentage_per_bin_graph = px.bar(count_by_category_and_bin, x='category', y='percentage', color='word_count_bins', barmode='stack').to_dict()

# plotly.io.from_json( get_graph_json(percentage_per_bin_graph) )

In [84]:
# Counts per category for the top 10 categories
# Gets the columns for the message categories
msg_category_cols = df.columns[4:]
# Counts of messages per category and gets the top 10 categories by count.
cat_count = df[ msg_category_cols ] \
            .sum() \
            .sort_values( ascending = False ) \
            .head( 10 ) \
            .reset_index() \
            .rename( columns = { 'index': 'category', 0: 'count' } )

# Count of messages per length bin
# Creating the bins and labels for the word count bins
bins = [ 0, 10, 20, 30, 40, 50, float('inf') ]
labels = [ '0-9', '10-19', '20-29', '30-39', '40-49', '50+' ]
# Calculates the word count per each message using the tokenize function
word_count = df['message'].apply( lambda msg: len( tokenize(msg) ) ).rename('word_count').reset_index(drop=True)
# Uses the pandas cut method to bin the word counts according to those created above
df['word_count_bin'] = pd.cut( word_count, bins=bins, labels=labels )
# word_count_bins = pd.cut( word_count, bins=bins, labels=labels )
# Counts the number of messages that fall into each bin
count_per_bin = df.groupby(['word_count_bin'], observed=False).agg( count=('word_count_bin', 'count') ).reset_index()

In [91]:
# Counts of word count bins per categories
# Gets the counts per bin and category
cat_wordcount_counts = df[['word_count_bin', *msg_category_cols]].groupby(['word_count_bin'], observed=False).sum().reset_index()
# Restructures the dataframe so that the category columns become one 'category' column
count_by_category_and_bin = pd.melt( cat_wordcount_counts, id_vars=['word_count_bin'], value_vars=msg_category_cols, var_name='category', value_name='count' )
# Gets the counts per category so that we can calculate the percentage later
total_counts = count_by_category_and_bin.groupby('category', observed=False)['count'].sum().reset_index().rename(columns={'count': 'total_count'})
# Merges the total counts per category back into the main dataframe
count_by_category_and_bin = count_by_category_and_bin.merge(total_counts, on='category')
# Calculates the percentage of messages per category and bin
count_by_category_and_bin['percentage'] = count_by_category_and_bin['count'] / count_by_category_and_bin['total_count'] * 100
# Rounds the percentage to 2 decimal places for better display
count_by_category_and_bin['percentage'] = count_by_category_and_bin['percentage'].apply( lambda x: round(x, 2) )
# Sorts the dataframe by category and word count bin
count_by_category_and_bin.sort_values(by=['category', 'word_count_bin'], ascending=[True, False], inplace=True)

# Graph 3: Count of messages per category and word count bin
percentage_per_bin_graph = px.bar(
    data_frame=count_by_category_and_bin, 
    x='category', 
    y='percentage', 
    title='Percentage of Messages per Category and Word Count Bin',
    labels={
        'x': 'category',
        'y': 'percentage'
    },
    color='word_count_bin', 
    barmode='stack'
)
plotly.io.from_json( get_graph_json(percentage_per_bin_graph) )